In [1]:
import s3fs
import nibabel as nib
import pandas as pd 
import matplotlib.pyplot as plt 
from nilearn import plotting 
import numpy as np

In [2]:
fs = s3fs.S3FileSystem(anon=True)

In [11]:
ll = fs.ls('natural-scenes-dataset/nsddata_betas/ppdata/subj01/MNI/betas_fithrf/betas_session0.nii.gz')
mask = fs.ls('natural-scenes-dataset/nsddata/ppdata/subj01/func1pt8mm/brainmask.nii.gz')

In [12]:
ll

[]

In [5]:

#for fname in ll: 
    #fs.get(fname, "tmp.nii.gz")
    #img = nib.load("tmp.nii.gz")

    # XXX do something with one file
fs.get(ll[0], "tmp.nii.gz")
fs.get(mask[0], "brainmask.nii.gz")
img = nib.load("tmp.nii.gz")
#brainmask_img = nib.load("brainmask.nii.gz")
#brainmask = brainmask_img.get_fdata()
#header = img.header

    # Append into total results:
    # results.append(XXX)

In [13]:
trials_info = pd.read_csv('../annotation_data/shared_trial_info.csv')
trials_sub1 = trials_info[((trials_info.SUBJECT == 1))]
trials_sub1.head()
len(trials_sub1)

3000

In [15]:
# 1.8 mm space functional data
# subj 1, session 1 
#trials = trials_sub1['trial_in_session'].tolist()

#beta_trials = []
#np.asarray(beta_trials)
for session in trials_sub1.SESSION.unique():
    if session < 10:
        ll = fs.ls(f'natural-scenes-dataset/nsddata_betas/ppdata/subj01/MNI/betas_fithrf/betas_session0{session}.nii.gz')
    else: 
        ll = fs.ls(f'natural-scenes-dataset/nsddata_betas/ppdata/subj01/MNI/betas_fithrf/betas_session{session}.nii.gz')
    fs.get(ll[0], "tmp.nii.gz")   
    img = nib.load("tmp.nii.gz")
    trials_sub_ses = trials_sub1[(trials_sub1.SESSION == session)]
    beta_trials = pd.DataFrame(columns = ["Betas", "Trial", "Session"])
    
    for count, t in enumerate(trials_sub_ses['trial_in_session'].tolist()):
        beta_trial = img.slicer[:,:,:,t].get_fdata()/300 #get trial 1
        #beta_trials = np.expand_dims(beta_trial,axis=3)
        beta_trials.at[count,'Betas'] = beta_trial
        beta_trials.at[count,'Trial'] = t
    
    beta_trials["Session"] = session
    if (session == 1):
        beta_trials.to_csv('betas_sub1.csv', index=False)
    else:
        beta_trials.to_csv('betas_sub1.csv', index=False, mode="a")
            #beta_trial = beta_trial[:,:,:,np.newaxis]
            #beta_trial = beta_trial[:,:,:,t]
            #beta_trials = np.append(beta_trials, beta_trial,axis=3)
    

KeyboardInterrupt: 

In [ ]:
print("session 1 beta, data shape:", beta_trials.shape)

In [ ]:
beta_trials.to_csv('betas_sub1_ses1.csv', index=False)

In [ ]:
# overlay beta map with brain mask
plotting.plot_stat_map(nib.Nifti1Image(beta_trial01, img.affine), # this function needs a 3D data file
                       bg_img=nib.Nifti1Image(brainmask, brainmask_img.affine), # load the brain mask as a background picture
                       cut_coords=(0, 0, 0), #use to select coordinates that you want to view
                       title="test",
                       vmax=5)

In [ ]:
type(beta_trials)

In [17]:
session_id = trials_sub1.SESSION.unique()

In [21]:
session_id[5]

6